In [75]:
# Start up the BQL service
import bql
import pandas as pd
import numpy as np

bq = bql.Service()   # <-- create BQL service object

In [76]:
region_country_mapping_df=pd.read_csv('region_country_mapping.csv')
master_region_list=list(region_country_mapping_df.REGION.unique())
master_region_list

['Asia',
 'AustraliaOceania',
 'EasternEurope',
 'MENA',
 'NorthAmerica',
 'SouthCentralAmerica',
 'SubsaharanAfrica',
 'WesternEurope']

In [77]:
rating_score_mapping_df=pd.read_csv('rating_score_mapping.csv')

In [ ]:
# def get_esg_score_for_isin(isin_list):
#     master_df=pd.DataFrame()
#     for input_isin in isin_list:
#         security = bq.univ.esgTicker(input_isin.strip()+' Corp')
#         esg_score = bq.func.value(bq.data.esg_disclosure_score()['value'],security)


#         req = bql.Request(input_isin, {
#             'ESG Disclosure Score':esg_score,
#         })
#         res = bq.execute(req)
#         df=res[0].df().reset_index().rename(columns = {'ID':'ID_ISIN'})[['ID_ISIN','ESG Disclosure Score']]
#         master_df=pd.concat([master_df,df])
#     return master_df

In [78]:
def get_esg_score_for_isin(isin_list):
    input_isin = [i.strip()+' Corp' for i in isin_list]
    security = bq.univ.esgTicker(input_isin)
    esg_score = bq.func.value(bq.data.esg_disclosure_score(fpt='a').fillna(limit='1',on='period')['value'],universe=security,mapby='lineage')


    req = bql.Request(input_isin, {
        'ESG Disclosure Score':esg_score,
    })
    res = bq.execute(req)
    df=res[0].df().reset_index()
    df['ID_ISIN']=[i.split()[0] for i in df['ID']]
    df=df.rename(columns={"ORIG_IDS:0":"ESG Ticker"})
    df=df[['ID_ISIN','ESG Ticker','ESG Disclosure Score']]
    return df

In [79]:
get_esg_score_for_isin(isin_list=['XS1759265264','US037833BA77'])

,ID_ISIN,ESG Ticker,ESG Disclosure Score
0,XS1759265264,656 HK Equity,73.201889
1,US037833BA77,AAPL US Equity,54.360729


In [ ]:
# # Get Region Country Mapping
# import bql
# bq = bql.Service()

# master_region_list=['Asia','AustraliaOceania','EasternEurope','MENA','NorthAmerica',
#                     'SouthCentralAmerica','SubsaharanAfrica','WesternEurope']
# NorthAmerica = bq.univ.members(master_region_list,type='Region')
# field = bq.data.id()

# req = bql.Request(NorthAmerica,field)
# res = bq.execute(req)
# region_country_mapping_df=res[0].df()
# region_country_mapping_df.to_csv('region_country_mapping_df.csv')

In [ ]:
# Get all Bonds for the ticker
import bql
import pandas as pd
bq = bql.Service()   # <-- create BQL service object

input_ticker = 'AAPL'
active_bonds = bq.univ.bondsuniv('Active')
bondsuniv = bq.univ.filter(active_bonds, bq.data.ticker() == input_ticker)

YTW = bq.data.yield_()['value']
name = bq.data.name()['value']
maturity_years = bq.data.maturity_years()['value']
ccy = bq.data.crncy()['value']
rating = bq.data.bb_composite()['value']

req = bql.Request(bondsuniv, {
    'Name':name,'YTW':YTW,'Time to Maturity':maturity_years,'Currency':ccy,'Rating':rating
})
res = bq.execute(req)
data = bql.combined_df(res).dropna()
data['Name']=data['Name']+' | '+data['Rating']+' | '+data['Currency']
data.head()

In [ ]:
import bqplot as bqp
import pandas as pd
import numpy as np

# Create some random data
dataframe = data.copy()

# Create Scales
scale_x = bqp.LinearScale()
scale_y = bqp.LinearScale()

# Create Mark
tooltip = bqp.Tooltip(fields=['x', 'y','name'],
                      labels=['Time to Maturity', 'YTW','Name'],
                      formats=['.3f', '.3f'])
mark_scatter = bqp.Scatter(x=dataframe['Time to Maturity'],
                           y=dataframe['YTW'],
                           names=dataframe['Name'], 
                            display_names=False,
                           tooltip=tooltip,
                           scales={'x': scale_x, 'y': scale_y})

# Create Axes
axis_x = bqp.Axis(scale=scale_x, label='Time to Maturity')
axis_y = bqp.Axis(scale=scale_y, orientation='vertical', label='YTW')

# Create Figure
figure = bqp.Figure(marks=[mark_scatter],
                    axes=[axis_x, axis_y],
                    animation_duration=500,
                    title=f"Yields for {input_ticker} Bonds",
                    title_style={'font-size': '22px'},
                    padding_x=0.05,
                    padding_y=0.05,
                    layout={'width': '100%', 'height': '500px'})

# Display the figure
figure


In [ ]:
# Get Region , Sector and rating for input bond
import bql
import pandas as pd
bq = bql.Service()   # <-- create BQL service object

input_ticker = 'AAPL'
input_maturity_mm_dd_yyyy = '08/20/2060'
input_maturity_mm_dd_yyyy=pd.to_datetime(input_maturity_mm_dd_yyyy).date()
active_bonds = bq.univ.bondsuniv('Active')
bondsuniv = bq.univ.filter(active_bonds, bq.func.and_(bq.data.ticker() == input_ticker , 
                                                      bq.data.maturity() == input_maturity_mm_dd_yyyy))

YTW = bq.data.yield_()['value']
name = bq.data.name()['value']
maturity_years = bq.data.maturity_years()['value']
issue_date = bq.data.issue_dt()['value']
sector = bq.data.bics_level_2_industry_group_name()['value']
country = bq.data.cntry_of_risk()['value']
rating = bq.data.bb_composite()['value']
ccy = bq.data.crncy()['value']

req = bql.Request(bondsuniv, {
    'Name':name,'YTW':YTW,'Time to Maturity':maturity_years,'Issue Date':issue_date,'Sector':sector,'Country of Risk':country,
    'Rating':rating,'Currency':ccy
})
res = bq.execute(req)
data = bql.combined_df(res)
data = data.dropna().sort_values(by="Issue Date").tail(1)
data['Region']=region_country_mapping_df[region_country_mapping_df.COUNTRY_CODE==data['Country of Risk'].values[0]]['REGION'].values[0]
data['Rating Score']=rating_score_mapping_df[rating_score_mapping_df.rating==data['Rating'].values[0]]['rating_score'].values[0]
data['Name']=data['Name']+' | '+data['Rating']+' | '+data['Currency']
data.head()

In [ ]:
# Get analytics for output bonds based on Region and Sector 
import bql
import pandas as pd
bq = bql.Service()   # <-- create BQL service object

bond_sector=data['Sector'].values[0]
bond_region=data['Region'].values[0]
bond_ccy=data['Currency'].values[0]
country_list_for_region=list(region_country_mapping_df[region_country_mapping_df.REGION==bond_region]['COUNTRY_CODE'].unique())
active_bonds = bq.univ.bondsuniv('Active')
bondsuniv = bq.univ.filter(active_bonds, bq.func.and_(bq.data.crncy()==bond_ccy, bq.func.and_(bq.data.cntry_of_risk().in_(country_list_for_region) , 
                                                      bq.data.bics_level_2_industry_group_name() == bond_sector)))

YTW = bq.data.yield_()['value']
name = bq.data.name()['value']
maturity_years = bq.data.maturity_years()['value']
country = bq.data.cntry_of_risk()['value']
rating=bq.data.bb_composite()['value']
ccy = bq.data.crncy()['value']

req = bql.Request(bondsuniv, {
    'Name':name,'YTW':YTW,'Time to Maturity':maturity_years,'Country of Risk':country,'Rating':rating,'Currency':ccy
})
res = bq.execute(req)
datas = bql.combined_df(res)
datas = datas.dropna()
datas['Region']=bond_region
datas['Sector']=bond_sector
datas['Name']=datas['Name']+' | '+datas['Rating']+' | '+datas['Currency']
datas.head()

In [ ]:
datas.Rating.unique()

In [ ]:
len(datas)

In [ ]:
import bqplot as bqp
import pandas as pd
import numpy as np

# Create some random data
dataframe = datas.copy()

# Create Scales
scale_x = bqp.LinearScale()
scale_y = bqp.LinearScale()

# Create Mark
tooltip = bqp.Tooltip(fields=['x', 'y','name'],
                      labels=['Time to Maturity', 'YTW','Name'],
                      formats=['.3f', '.3f'])
mark_scatter = [bqp.Scatter(x=dataframe['Time to Maturity'],
                           y=dataframe['YTW'],
                           names=dataframe['Name'], 
                            display_names=False,
                           tooltip=tooltip,
                           scales={'x': scale_x, 'y': scale_y}),
                bqp.Scatter(x=data['Time to Maturity'],
                           y=data['YTW'],
                           names=data['Name'], 
                            display_names=False,
                           tooltip=tooltip,
                            colors=['#FF5A00'],
                           scales={'x': scale_x, 'y': scale_y})
               ]

# Create Axes
axis_x = bqp.Axis(scale=scale_x, label='Time to Maturity')
axis_y = bqp.Axis(scale=scale_y, orientation='vertical', label='YTW')

# Create Figure
figure = bqp.Figure(marks=mark_scatter,
                    axes=[axis_x, axis_y],
                    animation_duration=500,
                    title=f"Yields for {bond_sector} Bonds in {bond_region} region",
                    title_style={'font-size': '22px'},
                    padding_x=0.05,
                    padding_y=0.05,
                    layout={'width': '100%', 'height': '500px'})

# Display the figure
figure


In [ ]:
# Get analytics for output bonds based on Region, Sector and Rating 
import bql
import pandas as pd
bq = bql.Service()   # <-- create BQL service object

bond_rating_score=data['Rating Score'].values[0]
bond_region=data['Region'].values[0]
bond_sector=data['Sector'].values[0]
bond_ccy=data['Currency'].values[0]
country_list_for_region=list(region_country_mapping_df[region_country_mapping_df.REGION==bond_region]['COUNTRY_CODE'].unique())
ratings_list_required=list(rating_score_mapping_df[(rating_score_mapping_df['rating_score']>=bond_rating_score-1) &
                                                   (rating_score_mapping_df['rating_score']<=bond_rating_score+1)
                                                  ]['rating'].unique())
active_bonds = bq.univ.bondsuniv('Active')
bondsuniv = bq.univ.filter(active_bonds, bq.func.and_(bq.data.crncy()==bond_ccy,
                                                      bq.func.and_(bq.func.and_(bq.data.cntry_of_risk().in_(country_list_for_region) , 
                                                      bq.data.bb_composite().in_(ratings_list_required)),
                                                     bq.data.bics_level_2_industry_group_name() == bond_sector)))

YTW = bq.data.yield_()['value']
name = bq.data.name()['value']
maturity_years = bq.data.maturity_years()['value']
country = bq.data.cntry_of_risk()['value']
rating=bq.data.bb_composite()['value']
ccy = bq.data.crncy()['value']

req = bql.Request(bondsuniv, {
    'Name':name,'YTW':YTW,'Time to Maturity':maturity_years,'Country of Risk':country,'Rating':rating,'Currency':ccy
})
res = bq.execute(req)
datas = bql.combined_df(res)
datas = datas.dropna()
datas['Region']=bond_region
datas['Sector']=bond_sector
datas['Name']=datas['Name']+' | '+datas['Rating']+' | '+datas['Currency']
datas.head()

In [ ]:
len(datas)

In [ ]:
import bqplot as bqp
import pandas as pd
import numpy as np

# Create some random data
dataframe = datas.copy()

# Create Scales
scale_x = bqp.LinearScale()
scale_y = bqp.LinearScale()

# Create Mark
tooltip = bqp.Tooltip(fields=['x', 'y','name'],
                      labels=['Time to Maturity', 'YTW','Name'],
                      formats=['.3f', '.3f'])
mark_scatter = [bqp.Scatter(x=dataframe['Time to Maturity'],
                           y=dataframe['YTW'],
                           names=dataframe['Name'], 
                            display_names=False,
                           tooltip=tooltip,
                           scales={'x': scale_x, 'y': scale_y}),
                bqp.Scatter(x=data['Time to Maturity'],
                           y=data['YTW'],
                           names=dataframe['Name'], 
                            display_names=False,
                           tooltip=tooltip,
                            colors=['#FF5A00'],
                           scales={'x': scale_x, 'y': scale_y})
               ]

# Create Axes
axis_x = bqp.Axis(scale=scale_x, label='Time to Maturity')
axis_y = bqp.Axis(scale=scale_y, orientation='vertical', label='YTW')

# Create Figure
figure = bqp.Figure(marks=mark_scatter,
                    axes=[axis_x, axis_y],
                    animation_duration=500,
                    title=f"Yields for {bond_sector} Bonds in {bond_region} region (Similar Rating)",
                    title_style={'font-size': '22px'},
                    padding_x=0.05,
                    padding_y=0.05,
                    layout={'width': '100%', 'height': '500px'})

# Display the figure
figure


In [80]:
#ESG Alternative
# Get Region , Sector , Currency and rating for input bond
import bql
import pandas as pd
bq = bql.Service()   # <-- create BQL service object

input_isin = 'US037833BA77'
security = input_isin.strip()+' Corp'

bondsuniv = bq.univ.bonds(security)

YTW = bq.data.yield_()['value']
name = bq.data.name()['value']
maturity_years = bq.data.maturity_years()['value']
issue_date = bq.data.issue_dt()['value']
sector = bq.data.bics_level_1_sector_name()['value'] #bics_level_2_industry_group_name
country = bq.data.cntry_of_risk()['value']
rating = bq.data.bb_composite()['value']
duration = bq.data.duration()['value']
ccy = bq.data.crncy()['value']


req = bql.Request(bondsuniv, {
    'Name':name,'YTW':YTW,'Time to Maturity':maturity_years,'Issue Date':issue_date,'Sector':sector,'Country of Risk':country,
    'Rating':rating,'Duration':duration,'Currency':ccy
})
res = bq.execute(req)
data = bql.combined_df(res)
data = data.sort_values(by="Issue Date").tail(1)
data['Region']=region_country_mapping_df[region_country_mapping_df.COUNTRY_CODE==data['Country of Risk'].values[0]]['REGION'].values[0]
data['Rating Score']=rating_score_mapping_df[rating_score_mapping_df.rating==data['Rating'].values[0]]['rating_score'].values[0]
data['ID_ISIN']=input_isin

esg_df=get_esg_score_for_isin(isin_list=[input_isin])
data=pd.merge(left=data.reset_index(),right=esg_df,how='left',left_on='ID_ISIN',right_on='ID_ISIN')
data.head()

,ID,Name,YTW,Time to Maturity,Issue Date,Sector,Country of Risk,Rating,Duration,Currency,Region,Rating Score,ID_ISIN,ESG Ticker,ESG Disclosure Score
0,BQ791790 Corp,AAPL 1.4 08/05/28,3.592449,6.069815,2021-08-05,Technology,US,AA+,5.653062,USD,NorthAmerica,35,US037833BA77,AAPL US Equity,54.360729


In [108]:
# Get list of similar ESG Bonds, based on sector, ccy, region, rating+/-1, Duration+/-3
 
bond_rating_score=data['Rating Score'].values[0]
bond_region=data['Region'].values[0]
bond_sector=data['Sector'].values[0]
bond_ccy=data['Currency'].values[0]
bond_duration=data['Duration'].values[0]
bond_esg_ticker=data['ESG Ticker'].values[0]
country_list_for_region=list(region_country_mapping_df[region_country_mapping_df.REGION==bond_region]['COUNTRY_CODE'].unique())
ratings_list_required=list(rating_score_mapping_df[(rating_score_mapping_df['rating_score']>=bond_rating_score-1) &
                                                   (rating_score_mapping_df['rating_score']<=bond_rating_score+1)
                                                  ]['rating'].unique())
active_bonds = bq.univ.bondsuniv('Active')
bondsuniv = bq.univ.filter(active_bonds, bq.func.and_(bq.data.duration().between(bond_duration-3, bond_duration+3),bq.func.and_(bq.data.crncy()==bond_ccy,
                                                      bq.func.and_(bq.func.and_(bq.data.cntry_of_risk().in_(country_list_for_region) , 
                                                      bq.data.bb_composite().in_(ratings_list_required)),
                                                     bq.data.bics_level_1_sector_name() == bond_sector))))

YTW = bq.data.yield_()['value']
name = bq.data.name()['value']
maturity_years = bq.data.maturity_years()['value']
country = bq.data.cntry_of_risk()['value']
rating=bq.data.bb_composite()['value']
duration = bq.data.duration()['value']
isin = bq.data.id_isin()['value']


req = bql.Request(bondsuniv, {
    'Name':name,'YTW':YTW,'Time to Maturity':maturity_years,'Country of Risk':country,'Rating':rating,'ID_ISIN':isin,
    'Duration':duration
})
res = bq.execute(req)
datas = bql.combined_df(res)
datas = datas.dropna()
datas['Region']=bond_region
datas['Sector']=bond_sector
datas['Currency']=bond_ccy

esg_df=get_esg_score_for_isin(isin_list=list(datas.ID_ISIN.unique()))
datas=pd.merge(left=datas.reset_index(),right=esg_df,how='left',left_on='ID_ISIN',right_on='ID_ISIN')
datas=datas[(datas['ESG Ticker']!=bond_esg_ticker) & datas['ESG Disclosure Score'].notnull()]
datas.head()

,ID,Name,YTW,Time to Maturity,Country of Risk,Rating,ID_ISIN,Duration,Region,Sector,Currency,ESG Ticker,ESG Disclosure Score
2,AM332115 Corp,MSFT 3.3 02/06/27,3.263340,4.574949,US,AAA,US594918BY93,4.044527,NorthAmerica,Technology,USD,MSFT US Equity,64.862694
3,QZ095291 Corp,MSFT 2.4 08/08/26,3.212380,4.076660,US,AAA,US594918BR43,3.714922,NorthAmerica,Technology,USD,MSFT US Equity,64.862694
9,QJ504145 Corp,MSFT 3 ⅛ 11/03/25,3.059935,3.315537,US,AAA,US594918BJ27,2.987945,NorthAmerica,Technology,USD,MSFT US Equity,64.862694


In [109]:
len(datas)

3

In [110]:
# Find Price correlation
base_bond=data.ID.values[0]
universe = list(np.append(data.ID.values,datas.ID.values)) #First bond is the base bond
date_range = bq.func.range('-1Y', '0D')
closing_prices = bq.data.px_last(dates=date_range, per='D').dropna()
req = bql.Request(universe, {'Price': closing_prices})
res = bq.execute(req)
df = res[0].df().reset_index().pivot(index='DATE', values='Price', columns='ID')     # <-- parse and pivot the dataframe in one line

# use the head() function to display the first 5 rows
df.corr()

ID,AM332115 Corp,BQ791790 Corp,QJ504145 Corp,QZ095291 Corp
ID,,,,
AM332115 Corp,1.000000,0.994531,0.997903,0.999064
BQ791790 Corp,0.994531,1.000000,0.992275,0.995594
QJ504145 Corp,0.997903,0.992275,1.000000,0.997888
QZ095291 Corp,0.999064,0.995594,0.997888,1.000000


In [115]:
corr_df=pd.DataFrame(df.corr()[base_bond])
corr_df.columns=['Price Correlation']
esg_alternatives_df=pd.concat([data,pd.merge(left=datas,right=corr_df,how='left',left_on='ID',right_index=True)])
esg_alternatives_df=esg_alternatives_df.set_index('ID').drop(['Issue Date', 'Rating Score'], axis=1).T

In [119]:
esg_alternatives_df

ID,BQ791790 Corp,AM332115 Corp,QZ095291 Corp,QJ504145 Corp
Name,AAPL 1.4 08/05/28,MSFT 3.3 02/06/27,MSFT 2.4 08/08/26,MSFT 3 ⅛ 11/03/25
YTW,3.592449,3.26334,3.21238,3.059935
Time to Maturity,6.069815,4.574949,4.07666,3.315537
Sector,Technology,Technology,Technology,Technology
Country of Risk,US,US,US,US
Rating,AA+,AAA,AAA,AAA
Duration,5.653062,4.044527,3.714922,2.987945
Currency,USD,USD,USD,USD
Region,NorthAmerica,NorthAmerica,NorthAmerica,NorthAmerica
ID_ISIN,US037833BA77,US594918BY93,US594918BR43,US594918BJ27


In [ ]:
import pandas as pd
import numpy as np
import bql
import math

bq = bql.Service()
#Select our bond
bond_1 = 'ZS7759443 Corp'
bond_2 = 'EC527035 Corp'

# Absolute dates, always in YYYY-MM-DD
bond_px1 = bq.data.PX_LAST(dates=bq.func.range('2019-12-07','2019-12-12'))
bond_px2 = bq.data.PX_LAST(dates=bq.func.range('2019-12-07','2019-12-12'))
req1 = bql.Request(bond_1, "bond_px1":bond_px1 <GO>)
req2 = bql.Request(bond_2, "bond_px2":bond_px2 <GO>)
res1 = bq.execute(req1)
res2 = bq.execute(req2)
px1_df = res1[0].df()
px2_df = res2[0].df()

px1 = px1_df["bond_px1"].tolist()
px2 = px2_df["bond_px2"].tolist()

px1_full = [x for x in px1 if math.isnan(x) == False]
px2_full = [i for i in px2 if math.isnan(i) == False]

corr = np.corrcoef(px1_full, px2_full)
corr